In [ ]:
#zadanie 13
from bs4 import BeautifulSoup
import pandas as pd
import random
from lxml import etree

ILE = 20000
MAMY = 100

def zamien(war):
    if(war < 1000):
        return 'DB00' + str(war)
    elif (war < 10000):
        return 'DB0' + str(war)
    else:
        return 'DB' + str(war)

with open("drugbank_partial.xml", 'r') as file:
    xml_page = file.read()


soup = BeautifulSoup(xml_page, 'xml')
data = soup.find_all(lambda lek: lek.name == 'drug' and lek.has_attr('xmlns'))

with open("drugbank_partial_and_generated.xml", "w", encoding="utf-8") as plik:
    plik.write(soup.prettify())

wartosci = {}
ostatni = 108
for lek in data:
    for tag in lek.children:
        #tagi = lek.find_all(True)
        if(tag.name is None):
            continue
        if(tag.name in wartosci):
            wartosci[tag.name].append(tag)
        else:
            wartosci[tag.name] = [tag]

ostatni += 1

drzewo = etree.parse("drugbank_partial_and_generated.xml")
baza = drzewo.getroot()


for i in range(ILE - MAMY):
    nowy_lek = etree.SubElement(baza, "drug", created="31-01-2025", type="biotech", updated="2024-01-02", xmlns="http://www.drugbank.ca")
    for klucz, lista in wartosci.items():
        if(klucz == "drugbank-id"):
            nowy_tag = etree.SubElement(nowy_lek, "drugbank-id", primary="true")
            nowy_tag.text = zamien(ostatni)
            ostatni+=1
            continue

        podtag = random.choice(lista)
        #print(ostatni-1)
        podtag2 = etree.fromstring(str(podtag))
        nowy_lek.append(podtag2)    
    baza.append(nowy_lek)

drzewo.write("drugbank_partial_and_generated.xml", pretty_print=True, xml_declaration = True)





In [1]:
# zadanie 1.
from bs4 import BeautifulSoup
import pandas as pd
with open("drugbank_partial_and_generated.xml", 'r') as file:
    xml_page = file.read()

soup = BeautifulSoup(xml_page, 'xml')
data = soup.find_all(lambda lek: lek.name == 'drug' and lek.has_attr('xmlns'))
wiersze = []

for lek in data:
    interakcje = []
    bloki = lek.find_all('food-interactions')
    
    for i in bloki:
        interakcje += i.find_all('food-interaction')
        
    lek_info = {
        'ID' : lek.find(lambda atr: atr.name == 'drugbank-id' and atr.has_attr('primary')).text,
        'Name' : lek.find('name').text,
        'Typ' : lek['type'],
        'Opis' : lek.find('description').text,
        'Postać' : lek.find('state').text,
        'Wskazania' : lek.find('indication').text,
        'Mechanizm działania': lek.find('mechanism-of-action').text,
        'Interakcje z pokarmami': [i.text for i in interakcje]    
    }
    wiersze.append(lek_info)

wynik = pd.DataFrame(wiersze)
wynik

: 

In [ ]:
#zadanie 3 wersja 1.
from bs4 import BeautifulSoup
import pandas as pd
with open("drugbank_partial.xml", 'r') as file:
    xml_page = file.read()

soup = BeautifulSoup(xml_page, 'xml')
data = soup.find_all(lambda lek: lek.name == 'drug' and lek.has_attr('xmlns'))

def znajdz_produkty(id):
    lek = None
    for lek_przeg in data:
        id_przeg = lek_przeg.find(lambda atr: atr.name == 'drugbank-id' and atr.has_attr('primary')).text
        if(id_przeg == id):
            lek = lek_przeg
            break

    if(lek == None):
        print("Podano błędne id.")
        return
    
    wiersze = []
    produkty = lek.find_all('products')
    
    for lista_prod in produkty:
        produkty2 = lista_prod.find_all('product')
        
        for produkt in produkty2:
            produkt_info = {
                "ID_leku" : id,
                "Nazwa_produktu " : produkt.find('name').text,
                "Producent" : produkt.find('labeller').text,
                "Kod" : produkt.find('ndc-product-code').text,
                "Postać": produkt.find('dosage-form').text,
                "Sposób aplikacji" : produkt.find('route').text,
                "Dawka" : produkt.find('strength').text,
                "Kraj" : produkt.find('country').text,
                "Agencja rejestrujące" : produkt.find('source').text
            }

            wiersze.append(produkt_info)
    return pd.DataFrame(wiersze)

znajdz_produkty('DB00001')